In [1]:
import pandas as pd
import sqlite3
import yfinance as yf
import requests
import json

In [2]:
start_date="2006-01-01"
end_date="2023-12-31"
tickers=["AAPL","MSFT",'SPY','XAUUSD.PEPPERSTONE','LCO.COMM']
data=pd.DataFrame()
conection=sqlite3.connect('database.db')
#tickers=['LCO.COMM']

In [3]:
def download_rename(data, name):
    name=name.replace('.','_')
    data.rename(columns={'o':f'{name}_Open'},inplace=True)
    data.rename(columns={'h':f'{name}_High'},inplace=True)
    data.rename(columns={'l':f'{name}_Low'},inplace=True)
    data.rename(columns={'c':f'{name}_Close'},inplace=True)
    data.rename(columns={'a':f'{name}_Adj Close'},inplace=True)
    data.rename(columns={'v':f'{name}_Volume'},inplace=True)
    return data

In [4]:
api_key='dd75440b5984491b9f3593d8cc275ed4'
for i in range(len(tickers)):
    #hystorical_daily_request=f'https://api.darqube.com/data-api/market-data/historical/daily/TSLA?token={api_key}&start_date={start_date}&end_date={end_date}'
    try:
        response=requests.get(f'https://api.darqube.com/data-api/market-data/historical/daily/{tickers[i]}?token={api_key}&start_date={start_date}&end_date={end_date}')
        response.raise_for_status()
        data_json=response.json()
        if data.empty:
            data=download_rename(pd.DataFrame(data_json),tickers[i])
        else:
            data=pd.merge(data,download_rename(pd.DataFrame(data_json),tickers[i]),on='t')
        #data=download_rename(pd.DataFrame(data_json),tickers[i])
        data.to_sql(f'{tickers[i]}',conection,index=False,if_exists='replace')
    except requests.exceptions.RequestException as e:
        print(f"API Error: {e}")



In [5]:
#data.to_sql('LCO',conection,index=False,if_exists='replace')

In [6]:
data.to_sql('stocks',conection,index=False,if_exists='replace')
data.shape
data

,t,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Volume,AAPL_Adj Close,MSFT_Open,MSFT_High,MSFT_Low,...,XAUUSD_PEPPERSTONE_Low,XAUUSD_PEPPERSTONE_Close,XAUUSD_PEPPERSTONE_Volume,XAUUSD_PEPPERSTONE_Adj Close,LCO_COMM_Open,LCO_COMM_High,LCO_COMM_Low,LCO_COMM_Close,LCO_COMM_Volume,LCO_COMM_Adj Close
0,1647475200,158.61,161.00,157.63,159.6529,75615400,159.6529,293.29,295.61,289.3700,...,1923.23000,1942.47000,162813.00,1942.47000,98.61,107.50,97.75,106.64,200947,106.64
1,1647561600,160.51,164.48,159.76,162.9926,123511700,162.9926,295.37,301.00,292.7300,...,1917.94000,1920.81000,125400.00,1920.81000,106.80,109.59,105.78,107.93,168906,107.93
2,1647820800,163.51,166.35,163.01,164.3842,95811400,164.3842,298.89,300.14,294.9000,...,1918.13000,1935.75000,129925.00,1935.75000,107.50,116.80,107.06,115.62,196494,115.62
3,1647907200,165.51,169.42,164.91,167.8035,81532000,167.8035,299.80,305.00,298.7700,...,1910.81000,1921.32000,137641.00,1921.32000,117.04,119.48,112.64,115.48,204659,115.48
4,1647993600,167.99,172.64,167.65,169.1851,98062700,169.1851,300.51,303.23,297.7201,...,1915.64000,1944.09000,128115.00,1944.09000,114.70,122.34,114.45,121.60,224526,121.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,1703203200,195.18,195.41,192.97,193.6000,37122800,193.6000,373.68,375.18,372.7100,...,2046.85498,2053.20002,185029.02,2053.20002,79.35,80.37,78.88,79.07,119189,79.07
444,1703548800,193.61,193.89,192.83,193.0500,28919300,193.0500,375.00,376.94,373.5000,...,2053.61998,2067.71002,75935.94,2067.71002,79.04,81.72,78.76,81.07,59735,81.07
445,1703635200,192.49,193.50,191.09,193.1500,48087700,193.1500,373.69,375.06,372.8100,...,2061.50998,2077.12002,113947.26,2077.12002,80.79,81.33,79.29,79.65,70385,79.65
446,1703721600,194.14,194.66,193.17,193.5800,34049900,193.5800,375.37,376.46,374.1600,...,2064.72498,2065.26502,161003.94,2065.26502,79.80,79.95,78.25,78.39,18782,78.39
